In [1]:
number_of_shots = 5

In [2]:
experiment_name = "2023.12.20-2_Mistral_7B_instruct_API_5-few-shots_RANDOM"

In [3]:
model_name = "Mistral_7B_instruct_API"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx").drop(columns=['Unnamed: 0'])
data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
    (data_sel_4_few_shot_examples.text_id==5614) |
    (data_sel_4_few_shot_examples.text_id==2089) |
    (data_sel_4_few_shot_examples.text_id==5665) |
    (data_sel_4_few_shot_examples.text_id==370)  |
    (data_sel_4_few_shot_examples.text_id==1422) #|
#     (data_sel_4_few_shot_examples.text_id==4329) |
#     (data_sel_4_few_shot_examples.text_id==4929) 

]

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
738,738,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000,96
1092,1092,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778,76
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000,136


In [11]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are {number_of_shots} examples of expected classification given an input text:

"""+few_shots+"""

Answer with the label only using the format:---LABEL---
"""


In [14]:
data_sel = data #.iloc[:5]

In [15]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": "mistralai/Mistral-7B-Instruct-v0.1",
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 9---
1 6099 0  ---SDG 12---
2 6471 0  ---SDG 16---
3 1786 0  ---SDG 9---
4 2143 0  ---SDG 8---
5 3377 0  ---SDG 16---
6 5589 0  ---SDG 9---
7 5431 0  ---SDG 8---
8 768 0  ---SDG 9---
9 2735 0  ---SDG 9---
10 1306 0  ---SDG 8---
11 3844 0  ---SDG 11---
12 1524 0  ---SDG 11---
13 4660 0  ---SDG 10---
14 5077 0  ---SDG 11---
15 995 0  ---SDG 11---
16 2166 0  ---SDG 16---
17 2512 0  ---SDG 16---
18 2191 0  ---SDG 3---
19 6073 0  ---SDG 11---
20 1376 0  ---SDG 13---
21 3889 0  ---SDG 3---
22 6576 0  ---SDG 1---
23 3912 0  ---SDG 12---
24 1565 0  ---SDG 8---
25 1033 0  ---SDG 1---
26 5653 0  ---SDG 9---
27 940 0  ---SDG 8---
28 3613 0  ---SDG 12---
29 1659 0  ---SDG 8---
30 848 0  ---SDG 13---
31 2157 0  ---SDG 13---
32 3604 0  ---SDG 8---
33 3940 0  ---SDG 3---
34 6228 0  ---SDG 4---
35 3148 0  ---SDG 11---
36 2298 0  ---SDG 3---
37 4226 0  ---SDG 10---
38 2335 0  ---SDG 3---
39 4832 0  ---SDG 3---
40 6153 0  ---SDG 8---
41 6772 0  ---SDG 11---
42 6340 0  ---SDG 11---
43 45

347 4308 5  ---SDG 5---
348 5020 5  ---SDG 3---
349 6718 5  ---SDG 8---
350 5449 5  ---SDG 5---
351 6086 5  ---SDG 4---
352 6002 5  ---SDG 5---
353 3703 5  ---SDG 5---
354 5927 5  ---SDG 3---
355 4613 5  ---SDG 4---
356 1510 5  ---SDG 5---
357 747 5  ---SDG 4---
358 630 5  ---SDG 4---
359 5175 5  ---SDG 16---
360 241 6  ---SDG 11---
361 372 6  ---SDG 6---
362 4479 6  ---SDG 6---
363 1568 6  ---SDG 6---
364 6194 6  ---SDG 6---
365 650 6  ---SDG 6---
366 4828 6  ---SDG 6---
367 3495 6  ---SDG 6---
368 2809 6  ---SDG 6---
369 4972 6  ---SDG 6---
370 3653 6  ---SDG 6---
371 379 6  ---SDG 6---
372 6287 6  ---SDG 7---
373 6555 6  ---SDG 6---
374 6626 6  ---SDG 6---
375 3955 6  ---SDG 6---
376 4253 6  ---SDG 6---
377 5477 6  ---SDG 6---
378 3247 6  ---SDG 6---
379 598 6  ---SDG 12---
380 4015 6  ---SDG 6---
381 961 6  ---SDG 6---
382 6717 6  ---SDG 6---
383 1321 6  ---SDG 6---
384 1347 6  ---SDG 6---
385 827 6  ---SDG 6---
386 4027 6  ---SDG 6---
387 6358 6  ---SDG 12---
388 3208 6  ---SDG 9-

685 1756 11  ---SDG 11---
686 3571 11  ---SDG 9---
687 3211 11  ---SDG 11---
688 3863 11  ---SDG 9---
689 4063 11  ---SDG 3---
690 5806 11  ---SDG 11---
691 5312 11  ---SDG 11---
692 3904 11  ---SDG 11---
693 3444 11  ---SDG 9---
694 1519 11  ---SDG 8---
695 1653 11  ---SDG 11---
696 1900 11  ---SDG 9---
697 2927 11  ---SDG 11---
698 3494 11  ---SDG 9---
699 3673 11  ---SDG 11---
700 3547 11  ---SDG 9---
701 5527 11  ---SDG 9---
702 4443 11  ---SDG 11---
703 926 11  ---SDG 11---
704 5272 11  ---SDG 3---
705 2640 11  ---SDG 9---
706 2678 11  ---SDG 7---
707 1725 11  ---SDG 11---
708 2798 11  ---SDG 10---
709 172 11  ---SDG 8---
710 2226 11  ---SDG 11---
711 1560 11  ---SDG 11---
712 3315 11  ---SDG 11---
713 5109 11  ---SDG 11---
714 5983 11  ---SDG 11---
715 4398 11  ---SDG 1---
716 3172 11  ---SDG 9---
717 148 11  ---SDG 11---
718 4937 11  ---SDG 11---
719 6039 11  ---SDG 7---
720 5667 12  ---SDG 12---
721 5041 12  ---SDG 12---
722 3004 12  ---SDG 12---
723 2110 12  ---SDG 12---
724 6

1005 339 16  ---SDG 16---
1006 4967 16  ---SDG 16---
1007 1794 16  ---SDG 5---
1008 1864 16  ---SDG 16---
1009 973 16  ---SDG 16---
1010 535 16  ---SDG 8---
1011 2370 16  ---SDG 11---
1012 2870 16  ---SDG 16---
1013 6330 16  ---SDG 16---
1014 2723 16  ---SDG 16---
1015 4695 16  ---SDG 16---
1016 6628 16  ---SDG 16---
1017 4563 16  ---SDG 16---
1018 6710 16  ---SDG 16---
1019 1947 16  ---SDG 10---


In [16]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [17]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

('827', 3451, 13, ' ---SDG 13---')
('607', 3374, 10, ' ---SDG 8---')
('765', 123, 12, ' ---SDG 12---')
('791', 4001, 13, ' ---SDG 13---')
('44', 1776, 0, ' ---SDG 10---')
('729', 1992, 12, ' ---SDG 12---')
('857', 4311, 14, ' ---SDG 14---')
('803', 6696, 13, ' ---SDG 7---')
('378', 3247, 6, ' ---SDG 6---')
('468', 6219, 7, ' ---SDG 7---')
('479', 4179, 7, ' ---SDG 7---')
('977', 6715, 16, ' ---SDG 16---')
('154', 3039, 2, ' ---SDG 1---')
('389', 5966, 6, ' ---SDG 6---')
('512', 6097, 8, ' ---SDG 4---')
('489', 1346, 8, ' ---SDG 8---')
('617', 4477, 10, ' ---SDG 1---')
('356', 1510, 5, ' ---SDG 5---')
('618', 1553, 10, ' ---SDG 1---')
('25', 1033, 0, ' ---SDG 1---')
('996', 4214, 16, ' ---SDG 16---')
('1004', 57, 16, ' ---SDG 16---')
('175', 6144, 2, ' ---SDG 12---')
('737', 4637, 12, ' ---SDG 12---')
('725', 5328, 12, ' ---SDG 12---')
('540', 1607, 9, ' ---SDG 9---')
('853', 1012, 14, ' ---SDG 13---')
('87', 375, 1, ' ---SDG 1---')
('275', 5343, 4, ' ---SDG 4---')
('682', 6047, 11, ' -

In [18]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [19]:
results_processed

,cnt,id,true_label,predicted_label
0,827,3451,13,---SDG 13---
1,607,3374,10,---SDG 8---
2,765,123,12,---SDG 12---
3,791,4001,13,---SDG 13---
4,44,1776,0,---SDG 10---
...,...,...,...,...
1015,594,359,9,---SDG 9---
1016,953,943,15,---SDG 12---
1017,448,5555,7,---SDG 7---
1018,648,3153,10,---SDG 8---


In [20]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 9---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 12---
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 9---
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 8---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---


In [21]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 9---,9
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 12---,12
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 9---,9
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 8---,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---,16


In [22]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
1,83,83,81,83,83,81,81,81,83,83,83,83,83
10,43,43,39,43,43,39,39,39,43,43,43,43,43
11,53,53,44,53,53,44,44,44,53,53,53,53,53
12,148,148,144,148,148,144,144,144,148,148,148,148,148
13,56,56,52,56,56,52,52,52,56,56,56,56,56
14,10,10,10,10,10,10,10,10,10,10,10,10,10
15,1,1,1,1,1,1,1,1,1,1,1,1,1
16,54,54,48,54,54,48,48,48,54,54,54,54,54
17,3,3,3,3,3,3,3,3,3,3,3,3,3


In [23]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [24]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.61      0.85      0.71        60
           2       0.89      0.55      0.68        60
           3       0.75      0.92      0.83        60
           4       0.77      0.88      0.82        60
           5       0.89      0.70      0.79        60
           6       0.98      0.77      0.86        60
           7       0.70      0.88      0.78        60
           8       0.32      0.82      0.46        60
           9       0.52      0.57      0.54        60
          10       0.44      0.32      0.37        60
          11       0.60      0.53      0.57        60
          12       0.39      0.97      0.56        60
          13       0.61      0.57      0.59        60
          14       1.00      0.17      0.29        60
          15       1.00      0.02      0.03        60
          16       0.80      0.72      0.75        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.